+# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

import warnings

warnings.filterwarnings("ignore")

In [95]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [4]:
recommender = MainRecommender(data_train_lvl_1)

In [5]:
recommender.get_als_recommendations(2375, N=5)

[899624, 1044078, 871756, 1106523, 832678]

In [6]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [7]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 937292, 934399, 15778319]

In [8]:
recommender.get_similar_users_recommendation(2375, N=5)

[10457044, 1124971, 945909, 1101502, 820612]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [9]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()

result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1 = result_lvl_1.query('user_id in @train_users')
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [10]:
# result_lvl_1 = result_lvl_1.head(255)

#### Зависимость от способа генерации кандидатов

In [11]:
# your_code
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=50) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual'], 50), axis=1).mean() * 100

4.823182317055945

In [12]:
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=50) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual'], 50), axis=1).mean() * 100

6.525657038145175

In [13]:
result_lvl_1['simr_it_rec'] = [recommender.get_similar_items_recommendation(i, N=50) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['simr_it_rec'], x['actual'], 50), axis=1).mean() * 100

3.4823830616485973

Лучший результат показал get_own_recommendations

#### Зависимость от k

get_own_recommendations

In [26]:
for k in (20, 50, 100, 200, 500):
    result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, k) for i in list(result_lvl_1['user_id'])]
    print("For k =", str(k) + ":", result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual'], k), axis=1).mean() * 100)

For k = 20: 3.928427679372909
For k = 50: 6.525657038145175
For k = 100: 9.604492955885034
For k = 200: 13.537278412833242
For k = 500: 18.20532455550868


get_als_recommendations

In [15]:
for k in (20, 50, 100, 200, 500):
    result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, k) for i in list(result_lvl_1['user_id'])]
    print("For k =", k, ":", result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual'], k), axis=1).mean() * 100)

For k =  20 : 2.9497297529200526
For k =  50 : 4.823182317055945
For k =  100 : 7.0450577921041315
For k =  200 : 9.814194864997553
For k =  500 : 14.630468518253279


get_similar_items_recommendation

In [16]:
for k in (20, 50, 100, 200, 500):
    result_lvl_1['simr_it_rec'] = [recommender.get_similar_items_recommendation(i, k) for i in list(result_lvl_1['user_id'])]
    print("For k =", k, ":", result_lvl_1.apply(lambda x: recall_at_k(x['simr_it_rec'], x['actual'], k), axis=1).mean() * 100)

For k =  20 : 1.8749589340911021
For k =  50 : 3.4823830616485973
For k =  100 : 5.4516028688080524
For k =  200 : 8.622188378696169
For k =  500 : 13.667900835663804


#### Вывод:

С ростом k растёт и качество отбора кандидатов, поэтому следует взять максимально возможное k

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [17]:
# your_code
data_train_lvl_2.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0
2107469,2021,40618753059,594,856060,1,1.77,443,-0.09,101,86,0.0,0.0
2107470,2021,40618753059,594,869344,1,1.67,443,-0.22,101,86,0.0,0.0
2107471,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,0.0,0.0


In [18]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=5))

In [19]:
users_lvl_2.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263]"
1,2021,"[950935, 1119454, 835578, 863762, 1019142]"


In [20]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1105426,1
0,2070,1097350,1
0,2070,879194,1
0,2070,948640,1


In [21]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [22]:
targets_lvl_2['target'].mean()

0.2867504835589942

In [66]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,average_weekly_quantity,average_dep_value
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,NaN,7.157367
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,NaN,1.432143


In [ ]:
# len(item_features)

In [47]:
# data_train_lvl_2[data_train_lvl_2['item_id'] == 869344]['sales_value'].mean()

In [48]:
# data_train_lvl_2[data_train_lvl_2['item_id'] == 869344]['sales_value'].unique()

In [51]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


### Генерация признаков

#### Для пользователя:

In [96]:
# генерация признаков:
# для пользователя:

# общая сумма покупок за период покупателя:
for i in user_features["user_id"]:
       user_features.loc[user_features['user_id'] == i, 'total_sum'] = data_train_lvl_2[data_train_lvl_2['user_id'] == i]['sales_value'].sum()

# средний чек:
for i in user_features["user_id"]:
    receipts = list()
    for j in data_train_lvl_2[data_train_lvl_2['user_id'] == i]['basket_id'].unique():
        receipts.append(data_train_lvl_2[data_train_lvl_2['user_id'] == i][data_train_lvl_2['basket_id'] == j]['sales_value'].sum())
    if len(receipts) != 0:
        user_features.loc[user_features['user_id'] == i, 'average_receipt'] = sum(receipts)/len(receipts)
    else: user_features.loc[user_features['user_id'] == i, 'average_receipt'] = 0

# самый популярный магазин:
for i in user_features["user_id"]:
    max_quantity = 0
    most_popular = 0
    for j in data_train_lvl_2[data_train_lvl_2['user_id'] == i]['store_id'].unique():
        new_m = len(data_train_lvl_2[(data_train_lvl_2['user_id'] == i)&(data_train_lvl_2['store_id'] == j)]['basket_id'].unique())
        if max_quantity < new_m:
            max_max_quantity = new_m
            most_popular = j
    user_features.loc[user_features['user_id'] == i, 'most_popular_store'] = j

In [97]:
user_features.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,total_sum,average_receipt,most_popular_store
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,341.78,48.825714,436.0
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,187.65,37.530000,359.0
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,304.14,50.690000,321.0


#### Для товаров:

In [98]:
# генерация признаков:
# для товаров:

# покупок в неделю:
for i in item_features["item_id"]:
    weeks = np.array(data_train_lvl_2[(data_train_lvl_2['item_id'] == i)]['week_no'].unique())
    if len(weeks) != 0:
        item_features.loc[item_features['item_id'] == i, 'average_weekly_quantity'] = data_train_lvl_2[(data_train_lvl_2['item_id'] == i)]['quantity'].sum()/len(weeks)
    else: item_features.loc[item_features['item_id'] == i, 'average_weekly_quantity'] = 0
        
# средняя цена в категории:
for i in item_features["commodity_desc"].unique():
    items_list = np.array(item_features[item_features["commodity_desc"] == i]["item_id"])
    item_features.loc[item_features["commodity_desc"] == i, 'average_cat_value'] = data_train_lvl_2[(data_train_lvl_2['item_id'].isin(items_list))]['sales_value'].sum()/len(items_list)
    
# соотношение цены товара к средней в категории:
for i in item_features['item_id']:
    item_features.loc[item_features['item_id'] == i, 'worth'] = data_train_lvl_2[data_train_lvl_2['item_id'] == i]['sales_value'].mean()/item_features[item_features['item_id'] == i]['average_cat_value']

In [115]:
item_features[item_features['average_weekly_quantity'] != 0]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,average_weekly_quantity,average_cat_value,worth
60,28116,69,GROCERY,Private,DRY MIX DESSERTS,GELATIN,.3 OZ,1.0,2.248187,0.146785
61,28117,69,GROCERY,Private,DRY MIX DESSERTS,GELATIN,.3 OZ,1.0,2.248187,0.151233
62,28143,69,GROCERY,Private,DRY MIX DESSERTS,GELATIN,.3 OZ,1.0,2.248187,0.146785
66,28186,69,GROCERY,Private,BAKING MIXES,BROWNIE MIX,10.25 OZ,1.0,4.512189,0.175081
74,28304,69,GROCERY,Private,SOUP,CONDENSED SOUP,10.75 OZ,1.0,7.499703,0.070669
...,...,...,...,...,...,...,...,...,...,...
91681,17330511,5042,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,2.0,1.432143,6.968579
91687,17382205,5456,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,1.0,1.432143,5.579052
91690,17383227,6422,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,,1.0,1.180511,3.803436
91695,17827644,906,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,32 OZ,1.0,1.432143,1.745636


#### Для пары товар-пользователь:

In [131]:
user_items_features = data_train_lvl_2[['user_id', 'item_id']].copy()

In [132]:
user_items_features = item_features[['item_id','commodity_desc']].copy().merge(user_items_features, on=['item_id'], how='right')

In [133]:
user_items_features

,item_id,commodity_desc,user_id
0,28116,DRY MIX DESSERTS,1415
1,28117,DRY MIX DESSERTS,1415
2,28143,DRY MIX DESSERTS,1415
3,28186,BAKING MIXES,486
4,28304,SOUP,1415
...,...,...,...
169706,17382205,NO COMMODITY DESCRIPTION,1872
169707,17383227,BOOKSTORE,2134
169708,17827644,NO COMMODITY DESCRIPTION,67
169709,17829232,CANDY - PACKAGED,1163


In [137]:
# генерация признаков:
# для пары товар-пользователь:

# (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
for i in item_features["commodity_desc"].unique():
    items_list = np.array(item_features[item_features["commodity_desc"] == i]["item_id"])
    weeks = len(data_train_lvl_2['week_no'].unique())
    mean_value = data_train_lvl_2[(data_train_lvl_2["item_id"].isin(items_list))]['quantity'].mean()/weeks
    for i in user_features['user_id']:
        user_pref = data_train_lvl_2[(data_train_lvl_2["user_id"] == i) & (data_train_lvl_2["item_id"].isin(items_list))]['quantity'].sum()/weeks
        user_items_features.loc[(user_items_features['commodity_desc'] == i)&(user_items_features['user_id'] == j), 'is_favorite_cat'] = user_pref - mean_value

KeyboardInterrupt: 

In [ ]:
# (Кол-во покупок юзером конкретной категории в неделю)/(Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
for i in item_features["commodity_desc"].unique():
    items_list = np.array(item_features[item_features["commodity_desc"] == i]["item_id"])
    weeks = len(data_train_lvl_2['week_no'].unique())
    mean_value = data_train_lvl_2[(data_train_lvl_2["item_id"].isin(items_list))]['quantity'].mean()/weeks
    for i in user_features['user_id']:
        user_pref = data_train_lvl_2[(data_train_lvl_2["user_id"] == i) & (data_train_lvl_2["item_id"].isin(items_list))]['quantity'].sum()/weeks
        user_items_features.loc[(user_items_features['commodity_desc'] == i)&(user_items_features['user_id'] == j), 'is_popular_cat'] = user_pref/mean_value

In [ ]:
# сколько пользователь 8 купил дессертов за все 6 недель (кол-во/неделю)
# items_list = np.array(item_features[item_features["commodity_desc"] == 'DRY MIX DESSERTS']["item_id"])
# data_train_lvl_2[(data_train_lvl_2["user_id"] == 8) & (data_train_lvl_2["item_id"].isin(items_list))]['quantity'].sum()/len(data_train_lvl_2['week_no'].unique())

In [ ]:
# среднее количество дессертов за все 6 недель, купленное каждым пользователем (кол-во/неделю)
# data_train_lvl_2[(data_train_lvl_2["item_id"].isin(items_list))]['quantity'].mean()/len(data_train_lvl_2['week_no'].unique())

In [ ]:
user_items_features

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 